# How to Track the Properties of Real Packets

**TARDIS** has the functionality to track the properties of the *RPackets* that are generated when running the Simulation. The `rpacket_tracker` can track all the interactions a packet undergoes & thus keeps a track of the various properties, a packet may have.<br>Currently, the `rpacket_tracker` tracks the properties of all the rpackets in the *Last Iteration of the Simulation*. It generates a `List` that contains the individual instances of `RPacketTracker`{`Numba JITClass`}, for storing all the interaction properties as listed below.

<html>
    <body>
        The properties that are tracked are as follows :
        <ol>
            <li><b>index</b> - Index of the Packet</li>
            <li><b>seed</b> - Seed of the Packet</li>
            <li><b>status</b> - Current Status for the Packet Interaction</li>
            <li><b>r</b> - Radius of the Current Shell</li> 
            <li><b>nu</b> - Packet's Frequency</li> 
            <li><b>mu</b> - Propagation Direction of the Packet (cosine of the angle the packet’s path makes with the radial direction)</li>
            <li><b>energy</b> - Energy of the Packet</li>
            <li><b>shell_id</b> - Current Shell Id where the Packet is present</li>
            <li><b>interaction_type</b> - Last Interaction type of the packet</li>
            <li><b>boundary_interaction</b> - Boundary interactions data in the format <b>(event_id, current_shell_id, next_shell_id)</b> of the packet</li>
        </ol>
    </body>
</html>

The data can be obtained in two ways i.e. `rpacket_tracker` and `rpacket_tracker_df`. The `rpacket_tracker` stores all the data for the interaction of the packets in a `list`, so it needs to accessed with a `list index` for each property for a particular `rpacket`. `rpacket_tracker_df` stores the data in a dataframe. Examples for the same are shown as follows. 

## How to Setup the Tracking for the RPackets?

**TARDIS**' `rpacket_tracker` is configured via the `YAML` file. This functionality of tracking the packets is turned **off**, by default. This is due to that fact that using this property, may slow down the execution time for the Simulation. An example configuration can be seen below for setting up the *tracking*:

```yaml
... 
montecarlo:
    ...
    tracking:
        track_rpacket: true
```

The `montecarlo` section of the **YAML** file now has a `tracking` sub section which holds the configuration properties for the `track_rpacket` & the `initial_array_length` (discussed later in the tutorial).

Let us see, the new `rpacket_tracker` in action.

In [1]:
from tardis.io.configuration.config_reader import Configuration

Iterations:          0/? [00:00<?, ?it/s]

Packets:             0/? [00:00<?, ?it/s]

In [2]:
# Reading the Configuration stored in `tardis_config_packet_tracking.yml` into config

config = Configuration.from_yaml("tardis_example.yml")

In [3]:
# Checking the `tracking` section via the Schema

config["montecarlo"]["tracking"]

{'track_rpacket': False, 'initial_array_length': 10}

In [4]:
# Setting `r_packet_tracking` to True to turn on the Tracking 

config["montecarlo"]["tracking"]["track_rpacket"] = True

In [5]:
config["montecarlo"]["tracking"]

{'track_rpacket': True, 'initial_array_length': 10}

In [6]:
from tardis import run_tardis
from tardis.io.atom_data import download_atom_data

In [7]:
download_atom_data('kurucz_cd23_chianti_H_He_latest')

Atomic Data kurucz_cd23_chianti_H_He_latest already exists in /home/runner/Downloads/tardis-data/kurucz_cd23_chianti_H_He_latest.h5. Will not download - override with force_download=True.


In [8]:
# Running the simulation from the config

sim = run_tardis(config, show_convergence_plots=False, show_progress_bars=False)

Auto-detected Jupyter notebook environment
Initializing panel with ipywidgets comms for Jupyter notebook


Auto-detected Jupyter notebook environment
Initializing panel with ipywidgets comms for Jupyter notebook


Initializing tabulator and plotly panel extensions for widgets to work


BokehModel(combine_events=True, render_bundle={'docs_json': {'b579f107-6c01-4440-85bf-7a05c86d2299': {'version…

Embedding the final state for Jupyter environments


Now, the `tracked` properties can be accessed via the `rpacket_tracker` attribute of the `sim.transport.transport_state` object. 

In [9]:
type(sim.transport.transport_state.rpacket_tracker)

numba.typed.typedlist.List

It can be seen from the above code, that the `sim.transport.transport_state.rpacket_tracker` is an instance of the `List` specifically *Numba Typed List*. The `RPacketTracker` class has the following structure for the properties : {More information in the **TARDIS API** for `RPacketTracker` class}

```python
# Basic structure for the RPacketTracker Class
class RPacketTracker:
   # Properties
    index
    seed
    status
    r
    nu
    mu
    energy
    shell_id
    interaction_type
    boundary_interaction
```

In [10]:
len(sim.transport.transport_state.rpacket_tracker)

100000

### Accessing properties through `rpacket_tracker`

To access these different properties, we may consider the following examples for the `rpacket_tracker`:
<br>In this Example, we are trying to access the properties of the packet at index `10`.<br>In a similar way, we can check for any property for any packet in the range of packets for the last iteration.

#### Accessing the `index` property for the packet with index = 10 :

In [11]:
sim.transport.transport_state.rpacket_tracker[10].index

10

#### Accessing the `seed` property for the packet with index = 10 :

In [12]:
sim.transport.transport_state.rpacket_tracker[10].seed

2729103521

#### Accessing the `status` property for the packet with index = 10 :

In [13]:
sim.transport.transport_state.rpacket_tracker[10].status

array([0, 0, 0, 0, 0, 0, 0, 1])

Thus, all other properties (`r`, `nu`, `mu`, `energy`, `shell_id`,`interaction_type`, `boundary_interaction`) can be accessed accordingly.

We can also see the total number of interactions of index `10` packet under went, with the following example:

In [14]:
len(sim.transport.transport_state.rpacket_tracker[10].shell_id)

8

<div class="alert alert-warning">

Warning

If we try to access `sim.transport.transport_state.rpacket_tracker` property when we have the `track_rpacket` property in the `tracking` subsection of `montecarlo` config, turned off as follows `config["montecarlo"]["tracking"]["track_rpacket"] = False`, it will return `None`.  Error will be raised if we try to access the properties i.e. `seed`, `index`, etc.
</div>

<div class="alert alert-info">

Note
    
When we initialise the `RPacketTracker()` class, the properties arrays {`index`, `seed`, `status`, etc} are allocated certain length based on the `initial_array_length` parameter that can be set via the `initial_array_length` property under `montecarlo -> tracking` section of the configuration. The default size of the array is `10`. This variable is important as the number of interactions a packet may have is variable, thus we need to allocate space dynamically. This variable is used to compute the size and expand the array such that the properties are able to hold these values for the packet interaction. Higher number, allocates more space initially leading to lesser times the arrays expands and vice versa. It can be set in the following manner `config["montecarlo"]["tracking"]["initial_array_length"] = {value}`.
</div>

### Accessing properties with `rpacket_tracker_df`

Also, the `tracked` properties can be accessed via the `rpacket_tracker_df` attribute of the `sim.transport` object. 

In [15]:
type(sim.transport.transport_state.rpacket_tracker_df)

pandas.core.frame.DataFrame

Here, all the properties of the rpackets discussed above are stored as a `Pandas.Dataframe` object. This makes accessing the properties of any rpacket more convenient and in a more organized way.

<html>
    <body>
        The Basic structure of the Dataframe is as follows:<br><br>
        Index Columns:
        <ol>
            <li><b>index</b> - Indicates the index number of the packet.</li>
            <li><b>step</b> - It is the step number that any particular packet is on, as it moves through the ejecta.</li>
        </ol>
        Properties:<br><br>
        These following properties are present as columns in the Dataframe. These are the same properties that were present in the <b>rpacket_tracker</b>
        <ol>
            <li><b>status</b></li>
            <li><b>seed</b></li>
            <li><b>r</b></li>
            <li><b>nu</b></li>
            <li><b>mu</b></li>
            <li><b>energy</b></li>
            <li><b>shell_id</b></li>
            <li><b>interaction_type</b></li>
        </ol>
    </body>
</html>

The entire dataframe, containing all the properties mentioned above, can be fetched as:

In [16]:
sim.transport.transport_state.rpacket_tracker_df

status        seed             r            nu        mu   energy  \
index step                                                                      
0     0          0  3338016338  1.235520e+15  1.736621e+15  0.839765  0.00001   
      1          0  3338016338  1.265887e+15  2.569710e+15 -0.608549  0.00001   
      2          0  3338016338  1.247165e+15  1.684576e+15  0.052209  0.00001   
      3          0  3338016338  1.247232e+15  1.682869e+15  0.025872  0.00001   
      4          0  3338016338  1.250482e+15  9.577694e+14 -0.281524  0.00001   
...            ...         ...           ...           ...       ...      ...   
99998 1          0   856604741  1.240969e+15  4.678746e+14  0.228747  0.00001   
      2          1   856604741  2.246400e+15  4.678746e+14  0.843086  0.00001   
99999 0          0   864415001  1.235520e+15  1.157499e+14  0.663583  0.00001   
      1          0   864415001  1.382024e+15  1.159135e+14  0.776785  0.00001   
      2          1   864415001  2.246400e+15  1.159135e+14  0.921894  0.00001   

            shell_id  interaction_type  
index step                              
0     0            0                -1  
      1            0                 2  
      2            0                 2  
      3            0                 2  
      4            0                 2  
...              ...               ...  
99998 1            0                 1  
      2           19                -1  
99999 0            0                -1  
      1            2                 1  
      2           19                -1  

[366901 rows x 8 columns]

To access these different properties in the dataframe, we may consider the following examples for the `rpacket_tracker_df`:
<br>In this Example, we are trying to access the properties of the packet at index `10`.<br>In a similar way, we can check for any property for any packet in the range of packets for the last iteration.

#### Accessing all the properties for the packet with index = 10:

In [17]:
sim.transport.transport_state.rpacket_tracker_df.loc[10]

,status,seed,r,nu,mu,energy,shell_id,interaction_type
step,,,,,,,,
0,0,2729103521,1.235520e+15,2.100602e+15,0.587433,0.000010,0,-1
1,0,2729103521,1.514322e+15,1.967100e+15,-0.707127,0.000010,5,1
2,0,2729103521,1.385949e+15,2.093353e+15,0.868696,0.000010,2,2
3,0,2729103521,1.708187e+15,2.754736e+15,0.357089,0.000010,9,2
4,0,2729103521,1.725576e+15,3.040719e+15,0.224209,0.000010,9,2
5,0,2729103521,1.751577e+15,1.680372e+15,0.226400,0.000010,10,2
6,0,2729103521,2.013038e+15,1.175266e+15,-0.054379,0.000009,15,2
7,1,2729103521,2.246400e+15,1.175266e+15,0.446484,0.000009,19,-1


#### Accessing the `energy` property for the packet with index = 10:


In [18]:
sim.transport.transport_state.rpacket_tracker_df.loc[10]["energy"]

step
0    0.000010
1    0.000010
2    0.000010
3    0.000010
4    0.000010
5    0.000010
6    0.000009
7    0.000009
Name: energy, dtype: float64

The above command returns a Pandas Series for the packet `#10` across different steps. To access any property at a particular step (for example at step `5`), the following commands can be used.

In [19]:
sim.transport.transport_state.rpacket_tracker_df.loc[10,5]["energy"]

np.float64(9.783241066555048e-06)

In [20]:
sim.transport.transport_state.rpacket_tracker_df.loc[10]["energy"][5]

np.float64(9.783241066555048e-06)

Thus, all other properties (`status`, `seed`, `r`, `nu`, `mu`, `shell_id`,`interaction_type`) can be accessed accordingly.